In [60]:
import scrapeKIPeople as skip

d = skip.pull_all()

In [62]:
d.keys()

dict_keys(["Andreea O'Connell", 'Karen Sveda', 'Jane Wilkinson', 'Matthew Vander Heiden', 'Nancy Hopkins', 'Francisco J. Sánchez-Rivera', 'Sangeeta Bhatia', 'Susan Hockfield', 'Yadira Soto-Feliciano', 'Ömer Yilmaz', 'Robert Langer', 'Daniel Anderson', 'Phillip Sharp', 'Darrell Irvine', 'Ram Sasisekharan', 'Michael Yaffe', 'Tyler Jacks', 'Angela Koehler', 'Stefani Spranger', 'Michael Cima', 'Frank B. Gertler', 'Dane Wittrup', 'Kristin Knouse', 'J. Christopher Love', 'Angela Belcher', 'Michael Hemann', 'Forest White', 'Frank Solomon', 'Michael Birnbaum', 'Scott Manalis', 'Jianzhu Chen', 'Paula Hammond', 'Jacqueline Lees', 'Richard O. Hynes', 'David Housman', 'Hojun Li', 'Joelle Straehla'])

In [6]:
from pathlib import Path
from bs4 import BeautifulSoup
fn = "/mnt/data/www/ki.mit.edu/people/administration.html"
soup = BeautifulSoup(Path(fn).read_text(), 'html.parser')

<tbody>
<tr class="tabletitle">
<th colspan="5">Directorship</th>
</tr>
<tr>
<th style="width: 251px;">Matthew Vander Heiden</th>
<td style="width: 533px;">Director</td>
<td>76-158</td>
<td>(617) 253-6403</td>
<td><a href="mailto:mvh@mit.edu">mvh@mit.edu</a></td>
</tr>
<tr>
<th style="width: 251px;">Darrell Irvine</th>
<td style="width: 533px;">Associate Director</td>
<td>76-261</td>
<td>(617) 452-4174</td>
<td><a href="mailto:djirvine@mit.edu">djirvine@mit.edu</a></td>
</tr>
<tr>
<th style="width: 251px;">Angela Koehler</th>
<td style="width: 533px;">Associate Director</td>
<td>76-361C</td>
<td>(617) 324-7631</td>
<td><a href="mailto:koehler@mit.edu">koehler@mit.edu</a></td>
</tr>
<tr>
<th style="width: 251px;">Jane Wilkinson</th>
<td style="width: 533px;">Executive Director</td>
<td>76-158</td>
<td>(617) 838-5118</td>
<td><a href="mailto:janewilk@mit.edu" target="_blank">janewilk@mit.edu</a></td>
</tr>
<tr>
<th style="width: 251px;">Andreea O'Connell</th>
<td style="width: 533px;">Di

In [54]:
def parse_administration_page(fn = "/mnt/data/www/ki.mit.edu/people/administration.html",
                              verbose=False):
    
    _table = BeautifulSoup(Path(fn).read_text(), 
                           'html.parser').select(
        'table[class=administrationtable] > tbody')[0]
    
    _admin = {}
    _group = ""
    for _r in _table.find_all('tr'):
        try:
            _th = _r.find('th').text.strip()
        except: 
            continue

        _td = []
        for _tdi in _r.find_all('td'):
            _td.append(_tdi.text.strip())
        if not _td:
            _group = _th
            _admin[_group] = {}
        else:
            _admin[_group][_th] = {
                "Title": _td[0],
                "Office": _td[1],
                "Phone": _td[2],
                "Email": _td[3]
            }
    return _admin

In [57]:
admin = parse_administration_page()

In [58]:
admin.keys()

dict_keys(['Directorship', 'General Administration', 'External Relations', 'Centers & Partnerships', 'Finance Administration', 'Core Facilities', 'Human Resources', 'Technology Services', 'Environmental Health and Safety & Building Operations'])

In [59]:
admin['Environmental Health and Safety & Building Operations']

{'Ryan Murphy': {'Title': 'Sr. Facilities & Operations Manager',
  'Office': '76-158',
  'Phone': '(617) 253-3168',
  'Email': 'rcmurphy@mit.edu'},
 'Mei Tran': {'Title': 'EHS Coordinator',
  'Office': '76-158',
  'Phone': '(617) 258-5123',
  'Email': 'emtran@mit.edu'}}

Now I've added this into `skip` so we can just pull using the function `skip.pull_administration_page()`.

In [1]:
import scrapeKIPeople as skip

admin = skip.pull_administration_page()

Now this is in a slightly different format than before since the previous dictionary had keys for each person and details about their work, whereas these have , but it a very informative one since it shows organizational structure of Administrators. Rather than combine with the entire list of faculty and investigators, let's lump the scientists together and then figure out a good way to merge Admin entities with with Scientist entities wherever possible.

In [2]:
admin, leadership, scientists = skip.pull_all()

In [4]:
admin.keys()

dict_keys(['Directorship', 'General Administration', 'External Relations', 'Centers & Partnerships', 'Finance Administration', 'Core Facilities', 'Human Resources', 'Technology Services', 'Environmental Health and Safety & Building Operations'])

In [5]:
leadership.keys()

dict_keys(["Andreea O'Connell", 'Karen Sveda', 'Jane Wilkinson'])

In [6]:
scientists.keys()

dict_keys(['Matthew Vander Heiden', 'Nancy Hopkins', 'Francisco J. Sánchez-Rivera', 'Sangeeta Bhatia', 'Susan Hockfield', 'Yadira Soto-Feliciano', 'Ömer Yilmaz', 'Robert Langer', 'Daniel Anderson', 'Phillip Sharp', 'Darrell Irvine', 'Ram Sasisekharan', 'Michael Yaffe', 'Tyler Jacks', 'Angela Koehler', 'Stefani Spranger', 'Michael Cima', 'Frank B. Gertler', 'Dane Wittrup', 'Kristin Knouse', 'J. Christopher Love', 'Angela Belcher', 'Michael Hemann', 'Forest White', 'Frank Solomon', 'Michael Birnbaum', 'Scott Manalis', 'Jianzhu Chen', 'Paula Hammond', 'Jacqueline Lees', 'Richard O. Hynes', 'David Housman', 'Hojun Li', 'Joelle Straehla'])

The reality is that I can really go to town with this since I already did the fine-tuning of each faculty that is stationed in the KI. The `people/faculty.html` actually contains quite a few additional Intramural Faculty. Let's grab in a way that's similar to the Administrator page.

In [7]:
def parse_faculty_page(fn = "/mnt/data/www/ki.mit.edu/people.html",
                              verbose=False):
    
    _table = BeautifulSoup(Path(fn).read_text(), 
                           'html.parser').select(
        'table[class=administrationtable] > tbody')[0]
    
    _admin = {}
    _group = ""
    for _r in _table.find_all('tr'):
        try:
            _th = _r.find('th').text.strip()
        except: 
            continue

        _td = []
        for _tdi in _r.find_all('td'):
            _td.append(_tdi.text.strip())
        if not _td:
            _group = _th
            _admin[_group] = {}
        else:
            _admin[_group][_th] = {
                "Title": _td[0],
                "Office": _td[1],
                "Phone": _td[2],
                "Email": _td[3]
            }
    return _admin

In [12]:
from pathlib import Path
from bs4 import BeautifulSoup

fn = "/mnt/data/www/ki.mit.edu/people.html"
soup = BeautifulSoup(Path(fn).read_text(), 'html.parser')
ssoup = soup.select('div[class=peoplefilt-group]')
len(ssoup)

3

In [37]:
for _ss in ssoup:
    _header = _ss.find('h2').text
    print(_header)
    for _p in _ss.find_all('div', class_="item person-teaser"):
        print(_p.find_all)
        print()
    print()

Intramural faculty



Daniel Anderson
Professor Anderson develops devices, materials, and vaccines to create customizable living and gene therapies for cancer, diabetes, and other applications.






Angela Belcher
Head of the Department of Biological Engineering
Professor Belcher seeks to understand and harness processes of nature to design materials and devices for cancer, energy, and the environment.






Sangeeta Bhatia
Director, Marble Center for Cancer Nanomedicine
Professor Bhatia integrates engineering, medicine, and biology to develop novel platforms for understanding, diagnosing, and treating cancer and other diseases.






Michael Birnbaum
Professor Birnbaum focuses on understanding and manipulating adaptive immune responses in cancer and infection.






Jianzhu Chen
Professor Chen seeks fundamental understandings of the immune system and their translation into immunotherapies for cancer and other diseases.






Michael Cima
Professor Cima develops materials, diagnostics

In [39]:
_ss = ssoup[0]

In [41]:
_header = _ss.find('h2').text
_header

'Intramural faculty'

In [43]:
_p = _ss.find_all('div', class_='item person-teaser')[0]
print(_p)

<div class="item person-teaser" role="article">
<div aria-hidden="true" class="image"><a href="people/faculty/daniel-anderson.html" tabindex="-1"><img alt="Daniel G. Anderson" class="image-style-person-thumbnail" height="260" loading="lazy" src="files/ki/styles/person_thumbnail/public/imce/people/intramural/Anderson-Daniel.jpg%3Fitok=09z4W-GI" width="208"/></a></div>
<div class="text">
<h3><a href="people/faculty/daniel-anderson.html">Daniel Anderson</a></h3>
<p class="interests">Professor Anderson develops devices, materials, and vaccines to create customizable living and gene therapies for cancer, diabetes, and other applications.</p>
</div>
</div>


In [66]:
_a = _p.find('h3').find('a')
_entry = {
    "Name": _a.text,
    "Interests": _p.find('p', class_='interests').text, 
    "Page": "https://ki.mit.edu/" + _a['href']
}
_entry

{'Name': 'Daniel Anderson',
 'Interests': 'Professor Anderson develops devices, materials, and vaccines to create customizable living and gene therapies for cancer, diabetes, and other applications.',
 'Page': 'https://ki.mit.edu/people/faculty/daniel-anderson.html'}

In [104]:
import re

_testr='on leave'
m = re.search('^(18|19|20)\d\d[\-](19|20|21)\d\d$',_testr)
if m:
    print(m.group(0))
else:
    print("Not accepted")

Not accepted


In [135]:
from pathlib import Path
from bs4 import BeautifulSoup
import re


def pull_people_page(fn = "/mnt/data/www/ki.mit.edu/people.html",
                     verbose = False):

    soup = BeautifulSoup(Path(fn).read_text(), 'html.parser').select('div[class=peoplefilt-group]')

    people_dict = {}
    for _ss in soup:
        _header = _ss.find('h2').text
        people_dict[_header] = {}
        for _p in _ss.find_all('div', class_="item person-teaser"):
            _a = _p.find('h3').find('a')
            _name = _a.text
            _note = False
            _passed = False
            if '(' in _name and ')' in _name:
                _ns = _name.split('(')
                _name = _ns[0]
                _note = _ns[1].split(')')[0]
                # test for life dates vs other notes
                _ld = re.search('^(18|19|20)\d\d[\-](19|20|21)\d\d$',_note)
                if _ld:
                    _passed = True
                    _birth = _note.split('-')[0]
                    _death = _note.split('-')[1]

            people_dict[_header][_name] = {
                "Interests": _p.find('p', class_='interests').text, 
                "Page": "https://ki.mit.edu/" + _a['href']
            }
            if _note:
                if _passed:
                    people_dict[_header][_name]['Life Dates'] = {
                        'Birth': _birth,
                        'Death': _death
                    }
                else:
                    people_dict[_header][_name]['Note'] = _note.capitalize()
                    
    return people_dict

In [136]:
people_dict = pull_people_page()

In [137]:
people_dict.keys()

dict_keys(['Intramural faculty', 'Extramural faculty', 'Emeritus and former faculty'])

In [138]:
people_dict['Emeritus and former faculty']

{'Angelika Amon': {'Interests': 'Cell biologist Angelika Amon studied molecular mechanisms governing chromosome segregation\xa0and the implications of chromosome mis-segregation for human diseases such as cancer.',
  'Page': 'https://ki.mit.edu/https://news.mit.edu/2020/angelika-amon-cell-biologist-pioneer-chromosome-imbalance-dies-53-1030',
  'Life Dates': {'Birth': '1967', 'Death': '2020'}},
 'Herman N. Eisen': {'Interests': 'Immunologist Herman Eisen uncovered fundamental biological processes, in particular interactions between the immune system and cancer cells.',
  'Page': 'https://ki.mit.edu/https://news.mit.edu/2014/herman-eisen-obituary-1113',
  'Life Dates': {'Birth': '1918', 'Death': '2014'}},
 'Frank B. Gertler': {'Interests': 'Biologist Frank B. Gertler explored the molecular mechanisms underlying tumor cell invasion and metastasis.',
  'Page': 'https://ki.mit.edu/people/faculty/frank-b-gertler.html'},
 'Nancy Hopkins': {'Interests': 'Geneticist Nancy Hopkins mapped RNA tum

In [139]:
people_dict['Extramural faculty']['Aviv Regev']

{'Interests': 'Professor Regev develops experimental and computational approaches to understand biological circuits, gene regulation, evolution, and alterations in cancer-causing genes.',
 'Page': 'https://ki.mit.edu/https://biology.mit.edu/people/aviv_regev',
 'Note': 'On leave'}

In [140]:
for _k in people_dict.keys():
    print(f"\n{_k}\n")
    for _kk in people_dict[_k].keys():
        print(f"\t- {_kk}")
    print()


Intramural faculty

	- Daniel Anderson
	- Angela Belcher
	- Sangeeta Bhatia
	- Michael Birnbaum
	- Jianzhu Chen
	- Michael Cima
	- Paula Hammond
	- Michael Hemann
	- Susan Hockfield
	- David Housman
	- Richard O. Hynes
	- Darrell Irvine
	- Tyler Jacks
	- Kristin Knouse
	- Angela Koehler
	- Robert Langer
	- Jacqueline Lees
	- J. Christopher Love
	- Scott Manalis
	- Francisco J. Sánchez-Rivera
	- Ram Sasisekharan
	- Phillip Sharp
	- Yadira Soto-Feliciano
	- Stefani Spranger
	- Matthew Vander Heiden
	- Forest White
	- Dane Wittrup
	- Michael Yaffe
	- Ömer Yilmaz


Extramural faculty

	- Regina Barzilay
	- Stephen Bell
	- Paul Blainey
	- Edward Boyden
	- Laurie Boyer
	- Christopher Burge
	- Eliezer Calo
	- Lindsay Case
	- Brandon DeKosky
	- Patrick Doyle
	- Elazer Edelman
	- Katie Galloway
	- Linda Griffith
	- Leonard Guarente
	- Anders Hansen
	- H. Robert Horvitz
	- Rudolf Jaenisch
	- Jeremiah A. Johnson
	- Roger Kamm
	- Amy Keating
	- Laura Kiessling
	- Eric Lander
	- Douglas Lauffenbur

And now I've put that function into `skip`, so I can call it and also incorporated it into the `skip.pull_all()`. After a session refresh, I can call it like this

In [1]:
import scrapeKIPeople as skip
skip.pull_people_page()

{'Intramural faculty': {'Description': 'Intramural faculty members have laboratories in the Koch Institute building.',
  'Daniel Anderson': {'Interests': 'Professor Anderson develops devices, materials, and vaccines to create customizable living and gene therapies for cancer, diabetes, and other applications.',
   'Page': 'https://ki.mit.edu/people/faculty/daniel-anderson.html'},
  'Angela Belcher': {'Interests': 'Professor Belcher seeks to understand and harness processes of nature to design materials and devices for cancer, energy, and the environment.',
   'Page': 'https://ki.mit.edu/people/faculty/angela-belcher.html'},
  'Sangeeta Bhatia': {'Interests': 'Professor Bhatia integrates engineering, medicine, and biology to develop novel platforms for understanding, diagnosing, and treating cancer and other diseases.',
   'Page': 'https://ki.mit.edu/people/faculty/sangeeta-bhatia.html'},
  'Michael Birnbaum': {'Interests': 'Professor Birnbaum focuses on understanding and manipulating a

In [2]:
admin, leadership, scientists, people = skip.pull_all()

In [7]:
for _k in people.keys():
    print(f"{_k}: {people[_k]['Description']}")
    print()
    for _k, _v in people[_k].items():
        if _k != 'Description':
            print(f"\t{_k}\n")
            print(f"\t\t{_v['Interests']}\n")
    print()

Intramural faculty: Intramural faculty members have laboratories in the Koch Institute building.

	Daniel Anderson

		Professor Anderson develops devices, materials, and vaccines to create customizable living and gene therapies for cancer, diabetes, and other applications.

	Angela Belcher

		Professor Belcher seeks to understand and harness processes of nature to design materials and devices for cancer, energy, and the environment.

	Sangeeta Bhatia

		Professor Bhatia integrates engineering, medicine, and biology to develop novel platforms for understanding, diagnosing, and treating cancer and other diseases.

	Michael Birnbaum

		Professor Birnbaum focuses on understanding and manipulating adaptive immune responses in cancer and infection.

	Jianzhu Chen

		Professor Chen seeks fundamental understandings of the immune system and their translation into immunotherapies for cancer and other diseases.

	Michael Cima

		Professor Cima develops materials, diagnostics, and treatments to im

Alright, finally time to start merging these more appropriately. Any additional pages might be an issue. Will `git` and lunch and get back to this.